In [1]:
### Lanelet 2

In [2]:
import lanelet2
import lanelet2.core as lncore
import os

In [6]:
# 1. creating (programmatically) and saving a simple lane

In [150]:
# simple init an empty map
lmap = lncore.LaneletMap()

# define a lane with a given left bound and right bound (and attribs)
lb = lncore.LineString3d(lncore.getId(),
                [lncore.Point3d(lncore.getId(), 4, 0, 0), lncore.Point3d(lncore.getId(), 4, 2, 0),
                 lncore.Point3d(lncore.getId(), 3, 3, 1), lncore.Point3d(lncore.getId(), 3, 4, 1)])
cb = lncore.LineString3d(lncore.getId(),
                [lncore.Point3d(lncore.getId(), 6, 0, 0), lncore.Point3d(lncore.getId(), 6, 2, 0),
                 lncore.Point3d(lncore.getId(), 5, 3, 1), lncore.Point3d(lncore.getId(), 5, 4, 1)])
rb = lncore.LineString3d(lncore.getId(),
                [lncore.Point3d(lncore.getId(), 8, 0, 0), lncore.Point3d(lncore.getId(), 8, 2, 0),
                 lncore.Point3d(lncore.getId(), 7, 3, 1), lncore.Point3d(lncore.getId(), 7, 4, 1)])
# first lane, normal
llet1 = lncore.Lanelet(lncore.getId(), lb, cb, lncore.AttributeMap({"type": "lanelet"}))
# its twin lane is inverted (opposite driving sense)
llet2 = lncore.Lanelet(lncore.getId(), cb, rb, lncore.AttributeMap({"type": "lanelet"})).invert()

lmap.add(llet1)
lmap.add(llet2)

# create a projector in London
projector = lanelet2.projection.UtmProjector(lanelet2.io.Origin(51.40, -0.04, 0.0))
ll_path = os.path.join(os.path.dirname(os.path.abspath('')), "data/llet_generated_map.osm")
lanelet2.io.write(ll_path, lmap, projector)

In [151]:
# 2. examing the result

In [152]:
# what's in a lanelet map? 
for s in dir(lmap):
    if "__" not in s:
        print s

add
areaLayer
laneletLayer
lineStringLayer
pointLayer
polygonLayer
regulatoryElementLayer


In [153]:
# let's look at the the lanelet layer 
lanes = [lane for lane in lmap.laneletLayer]
print("--- example of a lanelet ---\n\n%s" % lanes[0])
lane0 = lanes[0]
for s in dir(lane0):
    if "__" not in s:
        print "    * %s" % s

--- example of a lanelet ---

[id: 9217047218277095743, inverted, left id: 9217047218277095737 (inverted), right id: 9217047218277095732 (inverted)]
    * addRegulatoryElement
    * attributes
    * centerline
    * id
    * invert
    * inverted
    * leftBound
    * polygon2d
    * polygon3d
    * regulatoryElements
    * removeRegulatoryElement
    * rightBound
    * rightOfWay
    * speedLimits
    * trafficLights
    * trafficSigns


In [154]:
print("* lanelet left bound")
for p in llet1.leftBound:
    print("    %s " % p)
print("* lanelet left bound")
for p in llet1.rightBound:
    print("    %s " % p)

* lanelet left bound
    [id: 9217047218277095728 x: 4 y: 0 z: 0] 
    [id: 9217047218277095729 x: 4 y: 2 z: 0] 
    [id: 9217047218277095730 x: 3 y: 3 z: 1] 
    [id: 9217047218277095731 x: 3 y: 4 z: 1] 
* lanelet left bound
    [id: 9217047218277095733 x: 6 y: 0 z: 0] 
    [id: 9217047218277095734 x: 6 y: 2 z: 0] 
    [id: 9217047218277095735 x: 5 y: 3 z: 1] 
    [id: 9217047218277095736 x: 5 y: 4 z: 1] 


In [51]:
# 3. Now, open that map in JOSM
#       create a map with lanes and change the xml file to add lanelets

In [ ]:
#    if you have dawn linestrings in JOSM, the xml will be full of <way> and <nd> items
#       now create items <relation> and tag them "lanelet"
#
#    For example, 
#      <relation id="1010" version="1">
#        <member type="way" ref="-106947" role="left" />
#        <member type="way" ref="-106982" role="right" />
#            <tag k="type" v="lanelet" />
#            <tag k="subtype" v="road" />
#            <tag k="location" v="urban" />
#            <tag k="participant" v="vehicle:car" />
#       </relation>

In [126]:
osm_path = os.path.join(os.path.dirname(os.path.abspath('')), "data/tweacked_map.osm")
print("using OSM: %s (exists? %s)" % (osm_path, os.path.exists(osm_path)))
projector = lanelet2.projection.UtmProjector(lanelet2.io.Origin(41.15,15.10))
lmap, err_list = lanelet2.io.loadRobust(osm_path, projector) 
print("%d errors, %d lanes detected" % (len(err_list), len([l for l in lmap.laneletLayer])))

using OSM: /home/developer/workspace/data/tweacked_map.osm (exists? True)
0 errors, 2 lanes detected


In [130]:
lanes = [l for l in lmap.laneletLayer]

lane = lanes[0]
print("\nlane: %s" % (lane))
bnd = lane.leftBound
for i in range(0, len(bnd)):
    print("    %d -> id: %d (x: %.2f, y: %.2f)" % (i, bnd[i].id, bnd[i].x, bnd[i].y))

lane = lanes[1]
print("\nlane: %s" % (lane))
bnd = lane.rightBound
for i in range(0, len(bnd)):
    print("    %d -> id: %d (x: %.2f, y: %.2f)" % (i, bnd[i].id, bnd[i].x, bnd[i].y))


lane: [id: 1011, left id: -107500 (inverted), right id: -106982 (inverted)]
    0 -> id: -107527 (x: 556.63, y: 1245.03)
    1 -> id: -107525 (x: 602.48, y: 1293.51)
    2 -> id: -107523 (x: 628.98, y: 1331.65)
    3 -> id: -107521 (x: 696.99, y: 1287.35)
    4 -> id: -107519 (x: 612.04, y: 1204.30)
    5 -> id: -107513 (x: 615.66, y: 1187.72)
    6 -> id: -107560 (x: 621.97, y: 1175.62)
    7 -> id: -107511 (x: 627.39, y: 1164.42)
    8 -> id: -107509 (x: 623.80, y: 1153.21)
    9 -> id: -107507 (x: 625.18, y: 1127.65)
    10 -> id: -107505 (x: 648.77, y: 978.83)
    11 -> id: -107503 (x: 615.92, y: 976.54)
    12 -> id: -107501 (x: 604.66, y: 981.91)
    13 -> id: -107499 (x: 628.17, y: 896.75)
    14 -> id: -107498 (x: 637.71, y: 827.27)

lane: [id: 1010, left id: -106947, right id: -106982]
    0 -> id: -106980 (x: 628.73, y: 827.67)
    1 -> id: -106983 (x: 620.49, y: 889.09)
    2 -> id: -106984 (x: 609.02, y: 939.09)
    3 -> id: -106985 (x: 592.75, y: 996.20)
    4 -> id: -106

In [147]:
# bonus! the centerline

cline = lane.centerline
for i in range(0, 4):#len(cline)):
    print("    %d -> id: %d (x: %.2f, y: %.2f)" % (i, cline[i].id, cline[i].x, cline[i].y))
print("    ... etc ")

    0 -> id: 0 (x: 623.77, y: 827.01)
    1 -> id: 0 (x: 619.68, y: 857.49)
    2 -> id: 0 (x: 615.56, y: 888.20)
    3 -> id: 0 (x: 609.82, y: 913.20)
    ... etc 


In [ ]:
# 4. reading an existing OSM map

In [54]:
osm_path = os.path.join(os.path.dirname(os.path.abspath('')), "data/mapping_example.osm")
print("using OSM: %s (exists? %s)" % (osm_path, os.path.exists(osm_path)))
projector = lanelet2.projection.UtmProjector(lanelet2.io.Origin(49.00,8.42))
lmap, err_list = lanelet2.io.loadRobust(osm_path, projector) 
print("%d errors, %d lanes detected" % (len(err_list), len([l for l in lmap.laneletLayer])))

using OSM: /home/developer/workspace/data/mapping_example.osm (exists? True)
0 errors, 371 lanes detected


In [57]:
nmax = 10
print("* example of the laneletLayer (first %d)\n" % nmax)
for ln in lmap.laneletLayer:
    print ln
    nmax -= 1
    if nmax == 0:
        break

* example of the laneletLayer (first 10)

[id: 9178926741377113721, left id: 997750960516616982, right id: 2216115017118688772]
[id: 9037740909199276460, left id: 6937946819898808252, right id: 1729046099968656320]
[id: 8717970484406193818, left id: 5537827893167917386 (inverted), right id: 3128557769764614810]
[id: 8601933696747810962, left id: 2757326693420322496, right id: 2764641530780420290 (inverted)]
[id: 8396043010843852718, left id: 4860616454757881010 (inverted), right id: 2785499205350414700 (inverted)]
[id: 7599071200282796306, left id: 4406727741069666934, right id: 43190 (inverted)]
[id: 7326074532659563937, left id: 5214965862031460312, right id: 43270 (inverted)]
[id: 7195674799508775743, left id: 6960048458279195872, right id: 43178 (inverted)]
[id: 6980464299688733498, left id: 5375602761775609010, right id: 3440717602992035100 (inverted)]
[id: 6923355182620813640, left id: 5552362054548145838, right id: 2420601589665656822]


In [61]:
invlane = lmap.laneletLayer[8396043010843852718]
for a in invlane.attributes:
    print a

(location, urban)
(one_way, yes)
(region, de)
(subtype, road)
(type, lanelet)


In [155]:
lb = invlane.leftBound
print("left bound is a %s\n" % type(lb))
for a in lb.attributes:
    print a
print ("\noriginal")
for p in lb:
    print p
print("\nleft bound is inverted? %s" % lb.inverted())
print("\ninverted")
for p in lb.invert():
    print p

left bound is a <class 'lanelet2.core.LineString3d'>

(type, road_border)

original
[id: 39120 x: 327.67 y: 331.412 z: 0]
[id: 39118 x: 333.101 y: 325.933 z: 0]
[id: 39318 x: 336.352 y: 322.677 z: 0]

left bound is inverted? True

inverted
[id: 39318 x: 336.352 y: 322.677 z: 0]
[id: 39118 x: 333.101 y: 325.933 z: 0]
[id: 39120 x: 327.67 y: 331.412 z: 0]


In [66]:
# 5. Routing: go from a lanelet to another

In [94]:
# create a graph of roads
# ... and traffic rules (Germany)

trafficRules = lanelet2.traffic_rules.create(lanelet2.traffic_rules.Locations.Germany, 
                                             lanelet2.traffic_rules.Participants.Vehicle)
graph = lanelet2.routing.RoutingGraph(lmap, trafficRules)

In [76]:
# find all possible combination of routes that connect with a path

sp_map = dict()
for startLane in lmap.laneletLayer:
    for endLane in lmap.laneletLayer:
        if startLane.id != endLane.id:
            k = "%d_%d" % (startLane.id, endLane.id)
            rk = "%d_%d" % (endLane.id, startLane.id)
            sp = graph.shortestPath(startLane, endLane)
            if sp is not None:
                if k not in sp_map and rk not in sp_map:
                    sp_map[k] = sp
                
print("found %d connected lanes with shortest path" % len(sp_map))

found 11379 connected lanes with shortest path


In [77]:
# print 5 of them

nmax = 5
print("print the first %d" % nmax)
for k,v in sp_map.items():
    if nmax == 0:
        break
    print k
    nmax -= 1

print the first 5
6722104362058561355_6012398680329441872
5662180970588112254_6905469033316639457
329661501650965856_7326074532659563937
6771979691019578165_5872433480342781773
738566528952162269_1507837371260062763


In [95]:
# calculate the shortest path between two lanes

startLane = lmap.laneletLayer[6722104362058561355]
endLane = lmap.laneletLayer[6012398680329441872]

rt = graph.getRoute(startLane, endLane)
lpath = rt.shortestPath()

print ("found a path of %d lanes" % len(lpath))

found a path of 11 lanes


In [96]:
# show the path 

for i in range(0, len(lpath)):
    print("%d -> %s" % (1+i, lpath[i].leftBound))

1 -> [id: 43274, inverted point ids: 39264, 5620426488315080917, 4178248808544971958, 5891730753352617444, 5092834017878320153, 39010]
2 -> [id: 4472525114828817454 point ids: 39010, 39012]
3 -> [id: 5364935859541665514 point ids: 39012, 39014]
4 -> [id: 7771453129580263158 point ids: 39014, 823184189807104402]
5 -> [id: 4184469305594286820 point ids: 39360, 1311074889340970713, 3315398858022624714, 39362]
6 -> [id: 6110808963413952566 point ids: 39362, 5193105940432714582]
7 -> [id: 820629469327080082 point ids: 5193105940432714582, 39364, 623900778772383003]
8 -> [id: 43276 point ids: 623900778772383003, 39366, 3869800126136037116, 39368]
9 -> [id: 1455522156257738290 point ids: 39368, 8047403787857310581, 7778723713793902769]
10 -> [id: 6611545435134106938 point ids: 7778723713793902769, 39370, 4011328339141421088, 39372]
11 -> [id: 5687678308327519612 point ids: 39372, 39374]


In [97]:
# show the path (restricted to not change lane)

rempath = lpath.getRemainingLane(lpath[0])
for i in range(0, len(rempath)):
    print("%d -> %s" % (1+i, rempath[i].leftBound))

1 -> [id: 43274, inverted point ids: 39264, 5620426488315080917, 4178248808544971958, 5891730753352617444, 5092834017878320153, 39010]
2 -> [id: 4472525114828817454 point ids: 39010, 39012]
3 -> [id: 5364935859541665514 point ids: 39012, 39014]
4 -> [id: 7771453129580263158 point ids: 39014, 823184189807104402]
